# 如何联网查询数据

In [1]:
import os, sys
sys.path.append('..')

from utils.selenium_helper import *
from utils.str_helper import html_table_to_md
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Google Search

In [ ]:
url = 'https://www.google.com'
driver = init_driver()
try:
    open_page(driver, url)
    # Find the search box
    search_box = driver.find_element(By.NAME, 'q')
    # Type the search query
    search_query = '2024一季度生猪库存'
    search_box.send_keys(search_query)
    # Simulate hitting the 'Enter' key
    search_box.send_keys(Keys.RETURN)
    # Wait for results to load
    driver.implicitly_wait(5)  # seconds
    # Extract search results
    results = []
    search_results = driver.find_elements(By.CSS_SELECTOR, 'div.g')

    for result in search_results:
        title_element = result.find_element(By.CSS_SELECTOR, 'h3')
        title = title_element.text

        url_element = result.find_element(By.TAG_NAME, 'a')
        url = url_element.get_attribute('href')

        results.append({
            'url': url,
            'title': title
        })

    for item in results:
        print(item)
finally:
    close_driver(driver)


## 抽取东方财富-经营分析

In [ ]:
url = 'https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH688169&color=b#/jyfx'
driver = init_driver()
try:
    open_page(driver, url)
    table_html = get_table_html(driver, 'table.table')
    md_table = html_table_to_md(table_html)
    print(md_table)
finally:
    close_driver(driver)

## 下载巨潮报告

In [4]:
category = [
    {"key": "category_ndbg_szsh", "value": "年报"},
    {"key": "category_bndbg_szsh", "value": "半年报"},
    {"key": "category_yjdbg_szsh", "value": "一季报"},
    {"key": "category_sjdbg_szsh", "value": "三季报"},
    {"key": "category_yjygjxz_szsh", "value": "业绩预告"},
    {"key": "category_qyfpxzcs_szsh", "value": "权益分派"},
    {"key": "category_dshgg_szsh", "value": "董事会"},
    {"key": "category_jshgg_szsh", "value": "监事会"},
    {"key": "category_gddh_szsh", "value": "股东大会"},
    {"key": "category_rcjy_szsh", "value": "日常经营"},
    {"key": "category_gszl_szsh", "value": "公司治理"},
    {"key": "category_zj_szsh", "value": "中介报告"},
    {"key": "category_sf_szsh", "value": "首发"},
    {"key": "category_zf_szsh", "value": "增发"},
    {"key": "category_gqjl_szsh", "value": "股权激励"},
    {"key": "category_pg_szsh", "value": "配股"},
    {"key": "category_jj_szsh", "value": "解禁"},
    {"key": "category_gszq_szsh", "value": "公司债"},
    {"key": "category_kzzq_szsh", "value": "可转债"},
    {"key": "category_qtrz_szsh", "value": "其他融资"},
    {"key": "category_gqbd_szsh", "value": "股权变动"},
    {"key": "category_bcgz_szsh", "value": "补充更正"},
    {"key": "category_cqdq_szsh", "value": "澄清致歉"},
    {"key": "category_fxts_szsh", "value": "风险提示"},
    {"key": "category_tbclts_szsh", "value": "特别处理和退市"},
    {"key": "category_tszlq_szsh", "value": "退市整理期"}
]

In [2]:
from utils.tushare_helper import pro
df = pro.index_weight(index_code='000300.SH', trade_date='20240603')
df['symbol'] = df['con_code'].str.slice(0,6)
codes = df['symbol'].values

None


In [3]:
import time
import logging
from sys import stdout
from utils.cninfo import cninfo

logger = logging.getLogger('CnInfoReports')
logger.setLevel(logging.DEBUG)
sh = logging.StreamHandler(stdout)
sh.setFormatter(logging.Formatter(fmt='%(asctime)s %(levelname)s - %(message)s'))
logger.addHandler(sh)

#初始化并调用
instance = cninfo(skip_download_stock_json=True)
filter_template = {
    'market': 'szse',  #深沪京
    'tabName': 'fulltext',  #公告
    'plate': [],  #板块
    'category': [],  #公告分类
    'industry': [],  #行业
    'stock': [
        '600171',
    ],  #股票代码
    'searchkey': '',  #标题关键字
    'seDate': '2023-08-01~2024-05-18',  #起始时间
}

# 以10为步长遍历codes
for i in range(0, len(codes), 10):
    filter_copy = filter_template.copy()
    filter_copy['category'] = ['category_ndbg_szsh'] # 年度报告
    filter_copy['stock'] = codes[i:i+10]
    result = instance.query_announcements_info(filter_copy, download_pdf=True)
    time.sleep(5)

# filter_copy = filter_template.copy()
# filter_copy['stock'] = ['600171']
# filter_copy['category'] = ['category_ndbg_szsh']
# result = instance.query_announcements_info(filter_copy, download_pdf=True)

RemoteProtocolError: peer closed connection without sending complete message body (received 524288 bytes, expected 1136334)

In [ ]:
from utils.document_loader import auto_read_pdf
path = r"./data"
docs = auto_read_pdf(path)

In [ ]:
path = 'data/688169_石头科技_上海荣正企业咨询服务（集团）股份有限公司关于北京石头世纪科技股份有限公司2022年限制性股票激励计划第二个归属期符合归属条件之独立财务顾问报告_1220086495.pdf'

In [6]:
from langchain_community.document_loaders import UnstructuredPDFLoader
unstructure_kwargs = {
    "strategy": "fast",  # fast模式不对全图片的PDF进行识别
    "languages": ["eng", "chi"]
}
loader = UnstructuredPDFLoader(path, mode="elements", **unstructure_kwargs) 
docs = loader.load()

In [14]:
from unstructured.partition.pdf import partition_pdf
elements = partition_pdf(path)
for element in elements:
    print(element, type(element))

公司简称：石头科技 <class 'unstructured.documents.elements.Title'>
证券代码：688169 <class 'unstructured.documents.elements.Text'>
上海荣正企业咨询服务（集团）股份有限公司 <class 'unstructured.documents.elements.Title'>
关于 <class 'unstructured.documents.elements.Title'>
北京石头世纪科技股份有限公司 <class 'unstructured.documents.elements.Title'>
2022 年限制性股票激励计划 <class 'unstructured.documents.elements.Title'>
第二个归属期符合归属条件 <class 'unstructured.documents.elements.Title'>
之 <class 'unstructured.documents.elements.Title'>
独立财务顾问报告 <class 'unstructured.documents.elements.Title'>
2024 年 5 月 <class 'unstructured.documents.elements.Text'>
目录 <class 'unstructured.documents.elements.Title'>
一、释义 ..................................................................................................... 3 <class 'unstructured.documents.elements.Text'>
二、声明 ..................................................................................................... 4 <class 'unstructured.documents.elements.Text'>
三、基本假设 ........................................

## PPOCR识别文本

In [9]:
import os
import cv2
import numpy as np
from paddleocr import PPStructure,save_structure_res
from paddle.utils import try_import
from PIL import Image

ocr_engine = PPStructure(table=True, ocr=False, show_log=False)

save_folder = './output'
img_path = path

fitz = try_import("fitz")
imgs = []
with fitz.open(img_path) as pdf:
    for pg in range(0, pdf.page_count):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)

        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)

for index, img in enumerate(imgs):
    result = ocr_engine(img)
    save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0], index)
    for line in result:
        line.pop('img')
        print(line)

{'type': 'text', 'bbox': [511, 1349, 672, 1380], 'res': '', 'img_idx': 0}
{'type': 'title', 'bbox': [307, 969, 878, 1035], 'res': '', 'img_idx': 0}
{'type': 'header', 'bbox': [766, 183, 1006, 211], 'res': '', 'img_idx': 0}
{'type': 'header', 'bbox': [182, 182, 449, 211], 'res': '', 'img_idx': 0}
{'type': 'figure_caption', 'bbox': [572, 232, 621, 257], 'res': '', 'img_idx': 0}
{'type': 'footer', 'bbox': [574, 1563, 614, 1577], 'res': '', 'img_idx': 0}
{'type': 'reference', 'bbox': [178, 295, 1011, 585], 'res': '', 'img_idx': 0}
{'type': 'text', 'bbox': [179, 148, 290, 175], 'res': '', 'img_idx': 0}
{'type': 'table', 'bbox': [114, 196, 1075, 1464], 'res': {'cell_bbox': [[5.536468505859375, 4.514128684997559, 455.1169128417969, 4.871165752410889, 445.2604064941406, 65.16656494140625, 5.199908256530762, 62.30146789550781], [333.6568298339844, 7.851274490356445, 413.7137451171875, 8.146591186523438, 412.95013427734375, 68.21194458007812, 331.4310607910156, 66.62203979492188], [367.450897216

In [ ]:
import os
import cv2
from paddleocr import PPStructure,save_structure_res

table_engine = PPStructure(layout=False, show_log=True)

save_folder = './output'
img_path = path
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split('.')[0])

for line in result:
    line.pop('img')
    print(line)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.documents.base import Document
from langchain_core.pydantic_v1 import BaseModel, Field

class CleanDocument(BaseModel):
    result: str = Field(description="清理后的文本")
    removed: str = Field(description="被清理的文本或段落")

# model = Ollama(model="llama3", base_url='http://host.docker.internal:11434', temperature=0)
model = Ollama(model="qwen:32b", base_url='http://host.docker.internal:11434', temperature=0)
parser = PydanticOutputParser(pydantic_object=CleanDocument)
prompt = PromptTemplate(
    template="""
        去除文本中无法语义理解的段落,注意保留原文内容.
        原文内容：
        {text}
        生成格式：
        {format_instructions}
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
print(prompt)
prompt_and_model = prompt | model

def print_wrapped_text(text, width=80):
    for line in text.split('\n'):
        print('\n'.join([line[i:i+width] for i in range(0, len(line), width)]))

cleaned_doc = docs[:2]
for doc in cleaned_doc:
    cleaned_doc = Document(page_content=doc.page_content, metadata=doc.metadata)
    output = prompt_and_model.invoke({"text": cleaned_doc.page_content})
    print(output)
    break
    # cleaned_document = parser.invoke(output)
    # print_wrapped_text(cleaned_document, width=80)


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

model = Ollama(model="llama3", base_url='http://host.docker.internal:11434', temperature=0)

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke about germen."})
print(output)
parser.invoke(output)